In [ ]:
# NOTE
## The following C=C double-bonds in CBGA.pdb were manually placed using Avogadro
### C14 - C15
### C19 - C20

In [ ]:
%%bash
antechamber \
    -fi pdb -i structures/CBGA.pdb \
    -fo mol2 -o structures/CBGA.gaff.mol2 \
    -rn CBG1 -c bcc -at gaff2 -j 4 -pf y
parmchk2 \
    -i structures/CBGA.gaff.mol2 \
    -f mol2 -o CBGA.frcmod -s 2

In [ ]:
!tleap -f complex.in

In [ ]:
!sander -O \
    -i md.in -o md.out \
    -p complex.prmtop -c complex.inpcrd \
    -r md_qmmm.rst -x md_qmmm.mdcrd